## Getting ACS Data

This notebook goes through the steps for getting data from the American Community Survey (ACS) and joining it with our data from the Eviction Lab.

Steps:
1. Read Eviction Lab data and drop the years and counties we don't need
2. Obtain data from ACS API
3. Clean up ACS data - give columns meaningful names, etc.
4. Merge eviction data and ACS data to get our full dataset

In [1]:
import pandas as pd
import requests

In [2]:
evic_df = pd.read_csv("../data/raw/block-groups.csv")
print(evic_df.shape)

# keep rows for Cook County between 2012 and 2016
evic_df = evic_df.loc[evic_df["parent-location"] == "Cook County, Illinois"]
evic_df = evic_df.loc[(evic_df["year"] >= 2013) & (evic_df["year"] <= 2016)]
print(evic_df.shape)

(164747, 27)
(15972, 27)


In [3]:
evic_df.columns

Index(['GEOID', 'year', 'name', 'parent-location', 'population',
       'poverty-rate', 'renter-occupied-households', 'pct-renter-occupied',
       'median-gross-rent', 'median-household-income', 'median-property-value',
       'rent-burden', 'pct-white', 'pct-af-am', 'pct-hispanic', 'pct-am-ind',
       'pct-asian', 'pct-nh-pi', 'pct-multiple', 'pct-other',
       'eviction-filings', 'evictions', 'eviction-rate',
       'eviction-filing-rate', 'low-flag', 'imputed', 'subbed'],
      dtype='object')

In [4]:
# check distribution of GEOID values to see how many unique GEOIDs there are and confirm that
# each GEOID (block group) has five rows (years 2012-2016)
evic_df["GEOID"].value_counts().describe()

count    3993.0
mean        4.0
std         0.0
min         4.0
25%         4.0
50%         4.0
75%         4.0
max         4.0
Name: GEOID, dtype: float64

In [5]:
evic_df.head()

,GEOID,year,name,parent-location,population,poverty-rate,renter-occupied-households,pct-renter-occupied,median-gross-rent,median-household-income,...,pct-nh-pi,pct-multiple,pct-other,eviction-filings,evictions,eviction-rate,eviction-filing-rate,low-flag,imputed,subbed
8377,170310101001,2013,101.1,"Cook County, Illinois",435.0,18.92,159.0,67.23,785.0,NaN,...,0.0,3.22,0.0,14.0,4.0,2.52,8.83,0,0,0
8378,170310101001,2014,101.1,"Cook County, Illinois",435.0,18.92,161.0,67.23,785.0,NaN,...,0.0,3.22,0.0,19.0,4.0,2.48,11.78,0,0,0
8379,170310101001,2015,101.1,"Cook County, Illinois",435.0,18.92,164.0,67.23,785.0,NaN,...,0.0,3.22,0.0,13.0,3.0,1.83,7.92,0,0,0
8380,170310101001,2016,101.1,"Cook County, Illinois",435.0,18.92,167.0,67.23,785.0,NaN,...,0.0,3.22,0.0,12.0,2.0,1.20,7.19,0,0,0
8394,170310101002,2013,101.2,"Cook County, Illinois",1496.0,28.28,860.0,92.78,762.0,22537.0,...,0.0,3.07,0.0,36.0,20.0,2.33,4.19,0,0,0


In [6]:
# define components of URL for ACS API request
ACS_URL = "https://api.census.gov/data/2016/acs/acs5"
ACS_KEY = "bda4a7c6f7629b79fb63adf9b726149c173467bf" # TODO - make env variable

acs_vars = {
    "B23025_002E": "estimate_total_in_labor_force",
    "B23025_005E": "estimate_civilian_unemployed",
    "B19057_001E": "total_for_public_assistance_income",
    "B19057_002E": "with_public_assistance_income",
    "B25038_001E": "total_for_householder_tenure",
    "B25038_009E": "renter_occupied",
    "B25038_010E": "renter_moved_2015/2010_later",
    "B25038_011E": "renter_moved_2010-2014/2000-2009",
    "B25038_012E": "renter_moved_2000-2009/1990-1999",
    "B25038_013E": "renter_moved_1990-1999/1980-1989",
    "B25038_014E": "renter_moved_1980-1989/1970-1979",
    "B25038_015E": "renter_moved_1979/1969_earlier"
}

vars_list = [code for code, var in acs_vars.items()]
get_str = ",".join(sorted(vars_list))

params = {
    "get": get_str,
    "for": "block group:*",
    "in": ["state:17", "county:031"],
    "key": ACS_KEY
}

In [7]:
# make the request
results = requests.get(ACS_URL, params=params)

In [8]:
# look at the headers and first row of the JSON response
json_res = results.json()
json_res[:2]

[['B19057_001E',
  'B19057_002E',
  'B23025_002E',
  'B23025_005E',
  'B25038_001E',
  'B25038_009E',
  'B25038_010E',
  'B25038_011E',
  'B25038_012E',
  'B25038_013E',
  'B25038_014E',
  'B25038_015E',
  'state',
  'county',
  'tract',
  'block group'],
 ['312',
  '24',
  '394',
  '44',
  '312',
  '213',
  '0',
  '100',
  '113',
  '0',
  '0',
  '0',
  '17',
  '031',
  '010100',
  '1']]

In [9]:
# read_json did not work with results.json() so I'm passing it the URL constructed by requests
acs_df = pd.read_json(path_or_buf=results.url, orient="records")

# use the first row as the column names
acs_df.columns = acs_df.iloc[0]
acs_df = acs_df.reindex(acs_df.index.drop(0))

print(len(acs_df)) # confirm we have the same number of block groups here as in evic_df

3993


In [10]:
# Concatenate "state", "county", "tract", and "block group" to form a GEOID column
# in the ACS dataframe for joining with the evictions dataframe
acs_df["GEOID"] = acs_df["state"] + acs_df["county"] + acs_df["tract"] + acs_df["block group"]
acs_df["GEOID"] = acs_df["GEOID"].astype(int)
acs_df.head()

,B19057_001E,B19057_002E,B23025_002E,B23025_005E,B25038_001E,B25038_009E,B25038_010E,B25038_011E,B25038_012E,B25038_013E,B25038_014E,B25038_015E,state,county,tract,block group,GEOID
1,312,24,394,44,312,213,0,100,113,0,0,0,17,031,010100,1,170310101001
2,833,75,871,144,833,794,60,421,262,38,0,13,17,031,010100,2,170310101002
3,1061,14,1492,82,1061,883,34,594,204,51,0,0,17,031,010100,3,170310101003
4,619,0,675,101,619,396,36,221,117,22,0,0,17,031,010201,1,170310102011
5,1673,157,2279,248,1673,1278,118,738,323,68,31,0,17,031,010201,2,170310102012


In [11]:
# Rename columns to give them a more useful name than the ACS variable code
acs_df.rename(columns=acs_vars, inplace=True)
acs_df.head()

,total_for_public_assistance_income,with_public_assistance_income,estimate_total_in_labor_force,estimate_civilian_unemployed,total_for_householder_tenure,renter_occupied,renter_moved_2015/2010_later,renter_moved_2010-2014/2000-2009,renter_moved_2000-2009/1990-1999,renter_moved_1990-1999/1980-1989,renter_moved_1980-1989/1970-1979,renter_moved_1979/1969_earlier,state,county,tract,block group,GEOID
1,312,24,394,44,312,213,0,100,113,0,0,0,17,031,010100,1,170310101001
2,833,75,871,144,833,794,60,421,262,38,0,13,17,031,010100,2,170310101002
3,1061,14,1492,82,1061,883,34,594,204,51,0,0,17,031,010100,3,170310101003
4,619,0,675,101,619,396,36,221,117,22,0,0,17,031,010201,1,170310102011
5,1673,157,2279,248,1673,1278,118,738,323,68,31,0,17,031,010201,2,170310102012


In [12]:
#years = [2013, 2014, 2015, 2016, 2012]
years = [2013, 2014, 2015, 2016]
ACS_URL_years = {}
acs_complete = pd.DataFrame()
for year in years:
    ACS_URL_year = 'https://api.census.gov/data/' + str(year) + '/acs/acs5'
    results = requests.get(ACS_URL_year, params=params)
    acs_df = pd.read_json(path_or_buf=results.url, orient="records")
    acs_df.columns = acs_df.iloc[0]
    acs_df = acs_df.reindex(acs_df.index.drop(0))
    acs_df["GEOID"] = acs_df["state"] + acs_df["county"] + acs_df["tract"] + acs_df["block group"]
    acs_df["GEOID"] = acs_df["GEOID"].astype(int)
    acs_df.rename(columns=acs_vars, inplace=True)
    acs_df['year'] = year
    acs_complete = acs_complete.append(acs_df, ignore_index=True)
acs_complete

,total_for_public_assistance_income,with_public_assistance_income,estimate_total_in_labor_force,estimate_civilian_unemployed,total_for_householder_tenure,renter_occupied,renter_moved_2015/2010_later,renter_moved_2010-2014/2000-2009,renter_moved_2000-2009/1990-1999,renter_moved_1990-1999/1980-1989,renter_moved_1980-1989/1970-1979,renter_moved_1979/1969_earlier,state,county,tract,block group,GEOID,year
0,410,0,577,8,410,102,73,29,0,0,0,0,17,031,030200,3,170310302003,2013
1,711,18,1226,63,711,440,192,248,0,0,0,0,17,031,030200,4,170310302004,2013
2,640,9,744,100,640,421,132,192,81,16,0,0,17,031,030300,1,170310303001,2013
3,733,46,945,28,733,448,131,217,56,29,0,15,17,031,030300,2,170310303002,2013
4,785,114,833,166,785,556,213,256,55,32,0,0,17,031,030400,1,170310304001,2013
5,460,29,735,67,460,208,67,133,8,0,0,0,17,031,030400,2,170310304002,2013
6,592,0,765,27,592,278,87,136,11,44,0,0,17,031,030500,1,170310305001,2013
7,761,11,1128,61,761,451,152,206,93,0,0,0,17,031,030500,2,170310305002,2013
8,770,0,1018,55,770,469,197,215,44,13,0,0,17,031,030500,3,170310305003,2013
9,672,0,1245,86,672,362,107,230,0,0,0,25,17,031,030500,4,170310305004,2013


In [13]:
# Merge the ACS and evictions dataframes
merged_df = evic_df.merge(acs_complete, on=('GEOID', 'year'))

# Make sure the merged dataframe has the same number of rows as the evictions dataframe
print(len(merged_df))
merged_df.head()

15972


,GEOID,year,name,parent-location,population,poverty-rate,renter-occupied-households,pct-renter-occupied,median-gross-rent,median-household-income,...,renter_moved_2015/2010_later,renter_moved_2010-2014/2000-2009,renter_moved_2000-2009/1990-1999,renter_moved_1990-1999/1980-1989,renter_moved_1980-1989/1970-1979,renter_moved_1979/1969_earlier,state,county,tract,block group
0,170310101001,2013,101.1,"Cook County, Illinois",435.0,18.92,159.0,67.23,785.0,NaN,...,48,159,17,0,0,0,17,031,010100,1
1,170310101001,2014,101.1,"Cook County, Illinois",435.0,18.92,161.0,67.23,785.0,NaN,...,91,153,0,0,0,0,17,031,010100,1
2,170310101001,2015,101.1,"Cook County, Illinois",435.0,18.92,164.0,67.23,785.0,NaN,...,0,78,121,0,0,0,17,031,010100,1
3,170310101001,2016,101.1,"Cook County, Illinois",435.0,18.92,167.0,67.23,785.0,NaN,...,0,100,113,0,0,0,17,031,010100,1
4,170310101002,2013,101.2,"Cook County, Illinois",1496.0,28.28,860.0,92.78,762.0,22537.0,...,75,653,103,0,0,0,17,031,010100,2


In [14]:
# Exporting the result:
merged_df.to_csv('../data/work/block-groups_2012-2016_with-acs.csv')